In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pyprojroot import here
from llamabot import QueryBot
from IPython.display import display, Markdown

## Example 1: Read pre-built document index

We load a pre-made document index and query it.

In [ ]:
# Download the blog index from Dropbox using Requests.
# Taken from: https://stackoverflow.com/a/46005478
import requests

headers = {"user-agent": "Wget/1.16 (linux-gnu)"}  # <-- the key is here!
r = requests.get(
    "https://www.dropbox.com/s/xad2xf0n7isrhrz/blog_index.json?dl=0",
    stream=True,
    headers=headers,
)
blog_index = "/tmp/blog_index.json"
with open(blog_index, "wb") as f:
    for chunk in r.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

In [ ]:
bot = QueryBot(
    system_message="You are a digital twin of Eric Ma, the author of the blog posts you are about to query. Do not hallucinate any information.",
    saved_index_path=blog_index,
)
result = bot("How can I improve on my coding skills as a data scientist?")
# display(Markdown(result.response))
# result.print_response_stream()

In [ ]:
bot.chat_history[-1].content

In [ ]:
bot.source_nodes

In [ ]:
bot.chat_history[-1]

In [ ]:
display(Markdown(result.get_response().response))

In [ ]:
result.get_response()

In [ ]:
type(result)

In [ ]:
result.source_nodes

## Example 2: Loading documents directly

In [ ]:
from llamabot.file_finder import recursive_find

blog_posts = recursive_find(here() / "data" / "blog", file_extension=".lr")

bot = QueryBot(
    system_message="You are a digital twin of Eric Ma, the author of the blog posts that you will query.",
    doc_paths=blog_posts,
)

In [ ]:
from pathlib import Path

bot.save(Path.home() / "Downloads/blog_index.json")

In [ ]:
bot("What is your view on building a digital portfolio?")